# 자동화된 Machine Learning을 사용하여 분류 모델 학습

여러 종류의 기계 학습 알고리즘을 사용하여 모델을 학습시킬 수 있습니다. 그리고 특정 데이터 및 예측 요구 사항을 기준으로 가장 효율적인 알고리즘을 결정하기가 쉽지 않은 경우도 있습니다. 또한 정규화, 누락된 기능 대치 등의 기술을 사용해 학습 데이터를 전처리하면 모델의 예측 성능을 크게 개선할 수 있습니다. 요구 사항에 가장 적합한 모델을 찾으려면 여러 알고리즘 및 전처리 변환 조합을 적용해 보아야 할 수 있는데, 그러려면 시간이 많이 걸리며 컴퓨팅 리소스도 많이 필요합니다.

Azure Machine Learning을 사용하면 다양한 알고리즘 및 전처리 옵션을 사용하여 학습된 모델을 비교하는 과정을 자동화할 수 있습니다. [Azure Machine Learning Studio](https://ml/azure.com) 또는 Python SDK(v2)의 시각적 인터페이스를 사용하여 이 기능을 활용할 수 있습니다. Python SDK를 사용하면 자동화된 기계 학습 작업에 대한 설정을 더 잘 제어할 수 있지만 시각적 인터페이스를 사용하기가 더 쉽습니다.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 구독 ID, 리소스 그룹 이름 및 작업 영역 이름 등 식별자 매개 변수가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 데이터 준비

자동화된 Machine Learning용 학습 스크립트는 만들지 않아도 되지만, 학습 데이터는 로드해야 합니다. 

이 경우 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 

자동화된 기계 학습 작업에 입력으로 데이터 세트를 전달하려면 데이터가 테이블 형식이어야 하며 대상 열을 포함해야 합니다. 데이터를 테이블 형식 데이터 세트로 해석하려면 입력 데이터 세트가 **MLTable**이어야 합니다.

설정하는 동안 MLTable 데이터 자산이 이미 생성되었습니다. 데이터 페이지로 이동하여 **데이터** 자산을 탐색할 수 있습니다. 여기서는 이름 `diabetes-training-table` 및 버전을 `1`지정하여 데이터 자산을 검색합니다. 

In [ ]:
## 자동화된 기계 학습 작업 구성

이제 자동화된 기계 학습 실험을 구성할 준비가 되었습니다.

아래 코드를 실행하면 다음과 같은 자동화된 기계 학습 작업이 생성됩니다.

- 라는 컴퓨팅 클러스터 사용 `aml-cluster`
- 대상 열로 설정 `Diabetic`
- 기본 메트릭으로 설정 `accuracy`
- 총 학습 시간(분) 후 `60` 시간 초과 
- 최대 모델 학습 `5`
- 알고리즘을 사용하여 `LogisticRegression` 학습된 모델은 없습니다.

In [ ]:
## 자동화된 Machine Learning 작업을 실행합니다.

이제 실험을 실행할 준비가 되었으므로 자동화된 Machine Learning 실험을 실행해 보겠습니다.

> **참고**: 여기에는 시간이 다소 걸릴 수 있습니다.

작업이 실행되는 동안 Studio에서 모니터링할 수 있습니다.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.